In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json

In [ ]:
# open file
df = pd.read_excel('..\\corpus.xlsx', sheet_name='annotations')
df.columns = [c.lower() for c in df.columns]
df['id'] = df['code id'] + "-turn" + df['turn'].astype(str)
df = df[['id', 'text', 'das_gold', 'intent', 'argument', 'slot_names', 'slot_values']]
df.head(4)

,id,text,das_gold,intent,argument,slot_names,slot_values
0,ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa-turn1,hi,DS:opening; SOM:initGreeting,Nan,Nan,[],[]
1,ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa-turn2,"I don't know, try asking something like, ""What...",AutoF:autoNegative; DS:suggest,Nan,Nan,[],[]
2,ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa-turn3,describe the automaton,Ta:request,fsa-practical,Automata,[],[]
3,ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa-turn4,It is an automaton which accepts zero or more ...,Ta:answer,fsa-practical,Automata,"['alphabet','numberOfStates','states','initial...","[['0','1'],'3',['q0','q1','q2'],'q0',['q0'],[(..."


## Creo Train, Dev e Test set

In [26]:
# rename text as input
df = df.rename(columns={'text': 'input'})

In [27]:
df['output'] = df.apply(lambda row: json.dumps({
    'da': row['das_gold'],
    'et': row['intent'],
    'ar': row['argument'],
    'sn': row['slot_names'],
    'sv': row['slot_values']
}, ensure_ascii=False), axis=1)

In [28]:
# mantain only id, input and output
df = df[['id', 'input', 'output']]
df.head(1)

,id,input,output
0,ctyHWUgv9B_kHA6DNcrgMxadmJZJ1sHa-turn1,hi,"{""da"": ""DS:opening; SOM:initGreeting"", ""et"": ""..."


In [29]:
#foreach input in df mantain only the first
df = df.drop_duplicates(subset='input', keep='first')
len(df)

517

In [30]:
# Primo split: 80% training e 20% rimanente
train_df, dev_test_df = train_test_split(df, test_size=0.2, random_state=42)

# Secondo split: 50% dev e 50% test
dev_df, test_df = train_test_split(dev_test_df, test_size=0.5, random_state=42)

# Salva i set in file CSV senza apici extra
train_df.to_csv('train.csv', index=False, quoting=0)  # quoting=0 evita di mettere apici extra
dev_df.to_csv('dev.csv', index=False, quoting=0)
test_df.to_csv('test.csv', index=False, quoting=0)

print(f"Training set: {len(train_df)} record")
print(f"Dev set: {len(dev_df)} record")
print(f"Test set: {len(test_df)} record")

Training set: 413 record
Dev set: 52 record
Test set: 52 record
